# Filtering 

Some exoplanets are easier to observe than others. Let's walk through how we might filter down from all the transiting exoplanets in the NASA Exoplanet Archive into a `Population` for which we might want to consider trying to plan some observations.

In [ ]:
import exoatlas as ea 
ea.version()

Let's download a recent list of transiting exoplanets from the NASA Exoplanet Archive. We should, as always, be careful not to completely trust every detail of this database. For any particular planet, there's a possibility that details might be wrong or missing. For anything super crucial, it would be wise to double check!

In [ ]:
e = ea.TransitingExoplanets()
s = ea.SolarSystem()



Let's use a quick built-in summary to visualize how easy (or difficult) these planets are to observe with different methods. 

In [ ]:
import astropy.units as u 
ea.observable_summary([e,s]);

In [ ]:
ea.observable_summary([e,s], wavelength=0.5*u.micron, telescope_name='APO', observables=['depth', 'transmission']);

## Duration

In [ ]:
is_short = e.transit_duration < 3*u.hour

## Location on Sky

In [ ]:
from astropy.time import Time 
from astropy.coordinates import EarthLocation
import numpy as np
import matplotlib.pyplot as plt

midnight = Time('2024-09-21 00:00:00', scale='utc')

In [ ]:
ra_at_midnight = midnight.sidereal_time(kind='apparent',longitude='greenwich')

In [ ]:
is_north = e.dec > -10*u.deg
is_night = np.abs((e.ra - ra_at_midnight).wrap_at(180*u.deg)) < 6*u.hourangle

kw = dict(s=2, alpha=0.25)
plt.scatter(e.ra, e.dec, **kw)

potentially_observable = e[is_north * is_night]
plt.scatter(potentially_observable.ra, potentially_observable.dec, **kw)

plt.xlabel('Right Ascension (deg)')
plt.ylabel('Declination (deg)')
plt.xlim(360, 0)



## S/N for transmission spectroscopy
 
 

In [ ]:
signal = e.transmission_signal()
noise = e.depth_uncertainty(telescope='APO', wavelength=0.5*u.micron, dt=1*u.hour)
snr = e.transmission_snr(telescope='APO', wavelength=0.5*u.micron, dt=1*u.hour)

plt.scatter(noise, signal, c=snr)
plt.xscale('log'); plt.yscale('log')
plt.axis('scaled')
plt.xlabel('')
plt.ylim(1e-6, 1e-2); plt.xlim(1e-6, 1e-2)
plt.plot([1e-6, 1e-2], [1e-6, 1e-2])
plt.colorbar(label='S/N')

is_high_snr = snr > 5

## Put it all Together

In [ ]:
targets = e[is_high_snr*is_short*is_north*is_night]

In [ ]:
targets

In [ ]:
targets.name